In [ ]:
# for preprocessing html data
!pip install beautifulsoup4
# For RAG

!pip install langchainhub
!pip install chromadb
!pip install gpt4all
!pip install langchain==0.2.9
!pip install pypdf
!pip install langchain_community langchain_openai
!pip install transformers
!pip install openai
!pip install beautifulsoup4
!pip install faiss-cpu
!pip install langchain-openai
!pip install -U langchain-chroma

In [ ]:
from email import policy
from email.parser import BytesParser

In [ ]:
from langchain.docstore.document import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.embeddings import GPT4AllEmbeddings
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

## My email assistant 'Taala' ......



In [ ]:
from langchain_openai import OpenAI
import os
os.environ["OPENAI_API_KEY"] ="" # please use a valid key
llm = OpenAI(temperature=0.9)

In [ ]:
# text_file = open("last_mail.bin", "wb")
# text_file.write(b'85')
# text_file.close()

In [ ]:
# import os
# from google.colab import files

# # Zip the folder
# !zip -r chroma_db_email.zip chroma_db_email

# # Download the zipped file
# files.download('chroma_db_email.zip')


In [ ]:
# !unzip chroma_email_db.zip

In [ ]:
def splits(mail_content):

  doc = []

 # mail_number = int(last_mail_number.decode('utf-8'))+1

  for i in range(len(mail_content)):
    mail_number= int(mail_content[i].split(',')[0][10:-1])
    doci = Document(page_content=mail_content[i],
                  metadata={"source": "local","mail_number":mail_number})
    doc.append(doci)

  # split into different chunks
  # chunk_size and chunk_overlap are a hyperparameters we choose
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=20000, chunk_overlap=0)

  # split the documents
  all_splits = text_splitter.split_documents(doc) # Pass the list of documents 'doc'

  return all_splits


In [ ]:
def get_email_content(msg_list):
  for msgnum in msg_list:
        test = {}
        _, data = imap.fetch(msgnum, "(RFC822)")
        message = email.message_from_bytes(data[0][1])

        for part in message.walk():
            if part.get_content_type() == "text/plain":
                # Decode email payload
                payload = part.get_payload(decode=True)
                # Convert payload to string
                body = payload.decode(part.get_content_charset() or 'utf-8', errors='replace')

                # Clean unwanted content
                # Remove quoted-printable encoding artifacts
                body = re.sub(r'=([0-9A-Fa-f]{2})', lambda x: bytes.fromhex(x.group(1)).decode('latin-1'), body)

                # Optionally, remove headers and other unwanted text
                body = re.sub(r'^.*?Content-Type: text/plain;.*?$', '', body, flags=re.DOTALL)

                # Remove carriage returns
                body = body.replace('\r', '')

                # Use BeautifulSoup to extract text
                soup = BeautifulSoup(body, 'html.parser')
                body_text = soup.get_text().replace('"', '').replace("\n", " ").replace("\t", " ").strip()

                # Collect email details
                test = {
                    'Num': msgnum,
                    'message_id': message.get('Message-ID'),
                    'In_Reply_To': message.get('In-Reply-To'),
                    'References': message.get('References'),
                    'From': message.get('From'),
                    'To': message.get('To'),
                    'Date': message.get('Date'),
                    'Subject': message.get('Subject'),
                    'Body': body_text
                }

                content.append(str(test))

                break  # Exit after processing the first part

        # Use pprint to print the sixth email for verification
  # pprint(content)
  return content

In [ ]:
import imaplib
import email
from bs4 import BeautifulSoup
import re
from pprint import pprint
import os

file_path = 'last_mail.bin'
content = []
initial_flag = False
imap_server = "imap.gmail.com"
email_address = "" # enter gmailid
password = "" #enter password

# Connect to the IMAP server
imap = imaplib.IMAP4_SSL(imap_server, 993)
imap.login(email_address, password)
imap.select("Inbox")

# Search for all messages in the inbox
_, msgnums = imap.search(None, "ALL")

current_mail_list = msgnums[0].split()
new_mail_list = []
if os.path.exists(file_path):
    file_obj = open("last_mail.bin","rb")
    last_mail_number = file_obj.read()
    if current_mail_list[-1]==last_mail_number:
      print("No New Emails")
    else:
      ind_strt = current_mail_list.index(last_mail_number)+1
      new_mail_list = current_mail_list[ind_strt:]
      mail_content = get_email_content(new_mail_list)
      all_splits = splits(mail_content)
      text_file = open(file_path, "wb")
      text_file.write(new_mail_list[-1])
      print("overwriting to file")
      initial_flag = True
      text_file.close()

else:


    mail_content = get_email_content(current_mail_list)
    all_splits = splits(mail_content)
    text_file = open(file_path, "wb")
    text_file.write(current_mail_list[-1])
    print("writing to new file")
    text_file.close()


imap.close()
imap.logout()

overwriting to file


('BYE', [b'LOGOUT Requested'])

In [ ]:
mail_content

In [ ]:
embeddings = GPT4AllEmbeddings()

In [ ]:
from langchain_chroma import Chroma
if initial_flag:
    # Load the existing ChromaDB
    persistant_directory = "chroma_db_email"
    db = Chroma(persist_directory=persistant_directory, embedding_function=embeddings)

    # Add the new documents to the existing ChromaDB
    db.add_documents(documents=all_splits)

else:
    # If it's the first time, create a new ChromaDB
    db = Chroma.from_documents(documents=all_splits, embedding=embeddings, persist_directory="chroma_db_email")


In [ ]:
query = "What is regarding Jio ?"

In [ ]:
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 1})

In [ ]:
get_documents = retriever.invoke(query)

In [ ]:
combined_input = (
    "You are an email assistant and you are provided with some documents with email contents that might help you answer the question : "
    +query
    +"\n\nRelevant Documents: \n"
    + "\n\n".join([doc.page_content for doc in get_documents])
    + "\n\n Please provide an answer considering all the parameters in the provided documents. If the answer is not found, provide ''' Content Not found''' "
)



In [ ]:
from langchain.schema import SystemMessage, HumanMessage
messages = [
    SystemMessage(content ="You are a helpful assistant"),
    HumanMessage(content = combined_input)
]

In [ ]:
result = llm.invoke(messages)

In [ ]:
print(result)



Jio is a telecom service provider that highly prioritizes the security of its customers. Recently, there have been cases of scammers posing as Jio representatives and attempting to obtain sensitive information from customers. To avoid falling for such fraudulent activities, Jio has recommended its customers to never click on suspicious links or attachments, not respond to unsolicited emails, messages or calls asking for personal information, and not download apps that could provide remote access to their devices. Additionally, Jio will never ask its customers to download third-party apps and any communication from Jio will never ask them to click on unknown links outside of the official Jio website or MyJio app. It is also advised to regularly check bank and card statements for any irregularities and to keep devices updated and safe from unauthorized access. In case of any doubts, customers are recommended to log in to the MyJio app for relevant information. Overall, Jio takes great 